Importation des packages

On peut regarder la documentation de la base DVF sur le site suivant : 
https://apidf-preprod.cerema.fr/swagger/#/DVF%2B%20(acc%C3%A8s%20libre)/dvf_opendata_geomutations_list


In [2]:
import pandas as pd
import geopandas as gpd
import requests
from io import StringIO

Paramétrage de l'API

In [3]:
BASE_URL_API = "https://apidf-preprod.cerema.fr"
#code_insee, nom =  "59512", "Roubaix" #"75015", "Paris"
code_insee = "78220"
url = BASE_URL_API + f"/dvf_opendata/geomutations/?code_insee={code_insee}"
# %% 
print(url)

https://apidf-preprod.cerema.fr/dvf_opendata/geomutations/?code_insee=78220


In [4]:
response = requests.get(url)#, params=params)
# Vérification du succès de la requête
if response.status_code == 200:
    data = response.json()  # On récupère les données au format JSON
else:
    print(f"Erreur : {response.status_code}")
# %%
print(type(data))
print(data.keys())

<class 'dict'>
dict_keys(['type', 'count', 'next', 'previous', 'features'])


Data est un dictionnaire ; on va transformer son entrée "features" en un dataframe python

In [5]:
print(data["features"])

[{'id': 10179531, 'type': 'Feature', 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[1.893641, 48.715958], [1.893816, 48.71591], [1.894018, 48.715854], [1.894154, 48.715816], [1.894197, 48.715913], [1.894273, 48.715899], [1.89426, 48.715866], [1.894305, 48.715859], [1.894319, 48.715893], [1.894332, 48.715927], [1.89444, 48.716185], [1.894394, 48.716192], [1.894373, 48.716195], [1.894321, 48.716185], [1.894316, 48.716173], [1.894243, 48.716191], [1.894173, 48.716207], [1.894142, 48.716163], [1.894127, 48.716142], [1.894044, 48.716167], [1.894041, 48.716167], [1.89401, 48.716165], [1.893942, 48.716163], [1.893809, 48.716157], [1.893722, 48.716153], [1.893641, 48.715958]]]]}, 'properties': {'idmutinvar': '02770392cd0518ff5cfedef603fbe5e8', 'idopendata': '02770392cd0518ff5cfedef603fbe5e8', 'datemut': '2024-01-08', 'anneemut': 2024, 'coddep': '78', 'libnatmut': 'Vente', 'vefa': False, 'valeurfonc': '280740.00', 'nbcomm': 1, 'l_codinsee': ['78220'], 'nbpar': 1, 'l_idpar': ['78220000A

In [6]:
# Conversion en DataFrame pandas
df = pd.DataFrame(data["features"])
print(df.head())
print(df.columns)

         id     type                                           geometry  \
0  10179531  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[1....   
1  10179925  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[1....   
2  10179958  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[1....   
3  10180369  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[1....   
4  10180454  Feature  {'type': 'MultiPolygon', 'coordinates': [[[[1....   

                                          properties  
0  {'idmutinvar': '02770392cd0518ff5cfedef603fbe5...  
1  {'idmutinvar': '0fd6a5a5302c8f571b609895e6b8d0...  
2  {'idmutinvar': '7c302dd087b66bc3ed16e217713986...  
3  {'idmutinvar': '5a54c5ecc046b898a465647c288a07...  
4  {'idmutinvar': 'de094980803a4d6c80831ae8a99f1f...  
Index(['id', 'type', 'geometry', 'properties'], dtype='object')


In [7]:
df.head(1)

,id,type,geometry,properties
0,10179531,Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[1....",{'idmutinvar': '02770392cd0518ff5cfedef603fbe5...


On a importé un fichier geojson mais qu'on a traité comme un json classique ; le code ci-dessous vise à mieux prendre en compte cette dimension spatiale (et le côté "geo" du fichier geojson)

In [8]:
mut_gdf = gpd.read_file(StringIO(response.text))

# Afficher un aperçu des données
print(mut_gdf.head())

Skipping field l_codinsee: unsupported OGR type: 5
Skipping field l_idpar: unsupported OGR type: 5
Skipping field l_idparmut: unsupported OGR type: 5
Skipping field l_idlocmut: unsupported OGR type: 5


                         idmutinvar                        idopendata  \
0  02770392cd0518ff5cfedef603fbe5e8  02770392cd0518ff5cfedef603fbe5e8   
1  0fd6a5a5302c8f571b609895e6b8d0b0  0fd6a5a5302c8f571b609895e6b8d0b0   
2  7c302dd087b66bc3ed16e217713986a4  7c302dd087b66bc3ed16e217713986a4   
3  5a54c5ecc046b898a465647c288a0729  5a54c5ecc046b898a465647c288a0729   
4  de094980803a4d6c80831ae8a99f1f14  de094980803a4d6c80831ae8a99f1f14   

     datemut  anneemut coddep libnatmut   vefa valeurfonc  nbcomm  nbpar  \
0 2024-01-08      2024     78     Vente  False  280740.00       1      1   
1 2024-01-23      2024     78     Vente  False  120000.00       1      1   
2 2024-04-04      2024     78     Vente  False  465000.00       2      2   
3 2024-02-15      2024     78     Vente  False   90000.00       1      1   
4 2024-06-28      2024     78     Vente  False  498874.00       1      1   

   nbparmut    sterr  nbvolmut  nblocmut   sbati codtypbien  \
0         0     0.00         0         3 

Ok, mieux, mais il faudrait comprendre et régler ce qui s'affiche juste au dessus "Skipping field l_codinsee: unsupported OGR type: 5" (et les autres erreurs du même type)

In [9]:
# Afficher les colonnes
print(mut_gdf.columns)

Index(['idmutinvar', 'idopendata', 'datemut', 'anneemut', 'coddep',
       'libnatmut', 'vefa', 'valeurfonc', 'nbcomm', 'nbpar', 'nbparmut',
       'sterr', 'nbvolmut', 'nblocmut', 'sbati', 'codtypbien', 'libtypbien',
       'geometry'],
      dtype='object')


In [10]:
mut_gdf.head()

,idmutinvar,idopendata,datemut,anneemut,coddep,libnatmut,vefa,valeurfonc,nbcomm,nbpar,nbparmut,sterr,nbvolmut,nblocmut,sbati,codtypbien,libtypbien,geometry
0,02770392cd0518ff5cfedef603fbe5e8,02770392cd0518ff5cfedef603fbe5e8,2024-01-08,2024,78,Vente,False,280740.00,1,1,0,0.00,0,3,90.00,121,UN APPARTEMENT,"MULTIPOLYGON (((1.89364 48.71596, 1.89382 48.7..."
1,0fd6a5a5302c8f571b609895e6b8d0b0,0fd6a5a5302c8f571b609895e6b8d0b0,2024-01-23,2024,78,Vente,False,120000.00,1,1,0,0.00,1,0,0.00,102,BATI - INDETERMINE : Vente avec volume(s),"MULTIPOLYGON (((1.89657 48.71715, 1.89666 48.7..."
2,7c302dd087b66bc3ed16e217713986a4,7c302dd087b66bc3ed16e217713986a4,2024-04-04,2024,78,Vente,False,465000.00,2,2,2,1014.00,0,4,326.00,152,BATI MIXTE - LOGEMENT/ACTIVITE,"MULTIPOLYGON (((1.90259 48.72143, 1.90255 48.7..."
3,5a54c5ecc046b898a465647c288a0729,5a54c5ecc046b898a465647c288a0729,2024-02-15,2024,78,Vente,False,90000.00,1,1,0,0.00,0,3,34.00,121,UN APPARTEMENT,"MULTIPOLYGON (((1.89921 48.71781, 1.8996 48.71..."
4,de094980803a4d6c80831ae8a99f1f14,de094980803a4d6c80831ae8a99f1f14,2024-06-28,2024,78,Vente,False,498874.00,1,1,1,983.00,0,1,170.00,111,UNE MAISON,"MULTIPOLYGON (((1.90013 48.71268, 1.90016 48.7..."


Description des variables de ce dataframe :
- idmutinvar = identifiant unique entre les diff bases pour retrouver chaque mutation
- idopendata = identifiant de mutation utilisé depuis l'ouverture de données DVF
- datemut = date de signature de l'acte de vente
- anneemut = annee de signature du doc
- codedep
- libnatmut = Libellé de nature de mutation tel que défini dans la donnée source DVF 6 catégories : Vente, Vente en l'état futur d'achèvement, Adjudication, Echange, Vente terrain à bâtir et Expropriation.
- vefa = booléen selon si la vente est en VEFA (vente en l'état futur d'achèvement) ou non
- valeurfonc = valeur foncière en euro déclarée lors de la vente ; prix net vendeur, inclut la TVA mais n'inclut pas les frais de notaire, d'agence ou des meubles. 
- nbcomm = nombre de communes concernée par la mutation (i.e. sur lesquelles se trouve la parcelle cadastrale concernée par la mutation)
- nbpar = comme nbcomm mais pour les parcelles.
- nbparmut = nb de parcelles ayant muté (i.e. dont la pleine propriété a été transférée lors de la mutation)
- sterr = surface de terrain ayant muté (mutation = transfert de propriété, donc surface de terrain ayant muté = surface du terrain qui a été vendu)
- nbvolmut = variable obscure, flemme 
- nblocmut = nb de locaux ayant muté
- sbati = surface de l'ensemble du bati ayant muté
- codtypbien = typologie des biens ; on pourra s'en servir pour filter les maisons et appartements et virer tous les types de batiments bizarres (cf schéma ici) https://doc-datafoncier.cerema.fr/doc/dv3f/mutation/codtypbien
- libtypbien = comme codtypbien mais pour la base DV3F et non issu de juste DVF
- geometry

Les colonnes ont pour la plupart des types "object" qui nous empêchent de travailler : 

In [11]:
mut_gdf.dtypes

idmutinvar            object
idopendata            object
datemut       datetime64[ms]
anneemut               int32
coddep                object
libnatmut             object
vefa                    bool
valeurfonc            object
nbcomm                 int32
nbpar                  int32
nbparmut               int32
sterr                 object
nbvolmut               int32
nblocmut               int32
sbati                 object
codtypbien            object
libtypbien            object
geometry            geometry
dtype: object

On peut changer le type d'une colonne en un autre type à spécifier. Par exemple, pour changer 'sterr' en décimaux : 

In [12]:
mut_gdf['sterr']  = mut_gdf["sterr"].astype(float)

On peut vérifier :

In [13]:
mut_gdf.dtypes

idmutinvar            object
idopendata            object
datemut       datetime64[ms]
anneemut               int32
coddep                object
libnatmut             object
vefa                    bool
valeurfonc            object
nbcomm                 int32
nbpar                  int32
nbparmut               int32
sterr                float64
nbvolmut               int32
nblocmut               int32
sbati                 object
codtypbien            object
libtypbien            object
geometry            geometry
dtype: object